<a href="https://colab.research.google.com/github/10udCryp7/Speech-Practice/blob/main/notebooks/Music_Genre_Classification_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:06
🔁 Restarting kernel...


In [1]:
!conda install datasets[audio]

Channels:
 - conda-forge
Platform: linux-64
Solving environment: - \ | / - \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [2]:
!conda install evaluate

Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [79]:
from datasets import load_dataset
dataset = load_dataset("marsyas/gtzan", split = 'train')
dataset

Dataset({
    features: ['file', 'audio', 'genre'],
    num_rows: 999
})

In [80]:
dataset = dataset.train_test_split(test_size=0.1, shuffle=True, stratify_by_column='genre', seed = 42)
dataset

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

In [110]:
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification

model_id = "facebook/wav2vec2-base"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_id, do_normalize = True, return_attention_mask = True)

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/site-packages/transformers/configuration_utils.py:309: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [111]:
from datasets import Audio
dataset = dataset.cast_column("audio", Audio(sampling_rate=feature_extractor.sampling_rate))

In [112]:
id2label_fn = dataset['train'].features['genre'].int2str
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(dataset['train'].features['genre'].names))
}
label2id = {v: k for k, v in id2label.items()}

In [123]:
model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=len(id2label),
    label2id=label2id,
    id2label=id2label
)

/usr/local/lib/python3.11/site-packages/transformers/configuration_utils.py:309: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [124]:
max_dur = 30
def preprocess(examples):
  input_array = [example['array'] for example in examples['audio']]
  inputs = feature_extractor(input_array,
                             sampling_rate = feature_extractor.sampling_rate,
                             max_length = int(feature_extractor.sampling_rate * max_dur),
                             truncation = True,
                             return_attention_mask = True,
                             )
  return inputs

dataset_encoded = dataset.map(preprocess, batched=True, batch_size=100, remove_columns = ['audio', 'file'])

Map:   0%|          | 0/899 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [125]:
dataset_encoded = dataset_encoded.rename_column('genre', 'label')
dataset

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

In [120]:
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 16
gradient_accumulation_steps = 2
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    # fp16=True,
)

In [121]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["test"],
    processing_class=feature_extractor,
    compute_metrics=compute_metrics,

)

trainer.train()

Epoch,Training Loss,Validation Loss


In [61]:
dataset_encoded['train'][0]

{'label': 2,
 'input_values': [0.6964208483695984,
  0.887133777141571,
  0.7985947132110596,
  0.8532856702804565,
  0.8017014861106873,
  0.8537771105766296,
  0.8153052926063538,
  0.843326210975647,
  0.8103153109550476,
  0.8351865410804749,
  0.8118857741355896,
  0.8299083113670349,
  0.823677659034729,
  0.8357856869697571,
  0.8133158683776855,
  0.780391275882721,
  0.739014208316803,
  0.7184531688690186,
  0.7183743715286255,
  0.7305802702903748,
  0.7449216246604919,
  0.7419558167457581,
  0.7240121364593506,
  0.7004852294921875,
  0.6781031489372253,
  0.6583747863769531,
  0.6288518309593201,
  0.5889047980308533,
  0.5521029829978943,
  0.5464667081832886,
  0.5490078926086426,
  0.5366630554199219,
  0.5613428354263306,
  0.5741896629333496,
  0.5854488015174866,
  0.5812909603118896,
  0.6001155376434326,
  0.6510744690895081,
  0.6989845633506775,
  0.7303193211555481,
  0.7690942883491516,
  0.7904930710792542,
  0.8236662745475769,
  0.8578268885612488,
  0.8844